# GARCH Modeling

This notebook explores modeling volatility in financial time series using GARCH models. Volatility is the degree of variation of a trading price series over time, and it often exhibits clustering behavior known as conditional heteroskedasticity, where periods of high volatility tend to cluster together. The ARCH model introduced this concept by modeling variance as a function of past squared errors, and the GARCH model extends this by including past variances as well, providing a more flexible and widely used framework for volatility modeling.

## 1 Imports & Helpers

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from arch import arch_model

## 2 Load Data

We load a time series dataset, such as returns or price data, from a CSV file. We will compute log returns if the data is a price series.

In [4]:
# Load minute-level OHLC data and index by the timestamp
df = pd.read_csv(
    '/Users/mchildress/Active Code/ts_basics/data/bnbusdt_1m.csv',
    parse_dates=['open_time'],
    index_col='open_time'
)

# Pull out the 'close' column and make sure it's float
prices = df['close'].astype(float)

# Compute log returns on the price series
returns = np.log(prices).diff().dropna()

## 3 Fit ARIMA & check residuals

We fit a simple ARIMA model to capture the mean dynamics, then examine residuals to ensure no autocorrelation remains.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(returns, order=(1,0,1))
res = model.fit()

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.plot(res.resid)
plt.title('ARIMA Residuals')
plt.subplot(122)
plot_acf(res.resid, ax=plt.gca(), lags=40)
plt.title('ACF of Residuals')
plt.tight_layout()
plt.show()

/Users/mchildress/.pyenv/versions/3.11.12/envs/dream/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mchildress/.pyenv/versions/3.11.12/envs/dream/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/mchildress/.pyenv/versions/3.11.12/envs/dream/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


## 4 Test for ARCH effects

We plot the autocorrelation of squared residuals to check for volatility clustering, a sign of ARCH effects.

In [ ]:
plt.figure(figsize=(6,4))
plot_acf(res.resid**2, lags=40)
plt.title('ACF of Squared Residuals')
plt.show()

## 5 Fit GARCH(1,1)

We fit a GARCH(1,1) model to capture volatility clustering and time-varying variance.

In [ ]:
garch = arch_model(res.resid, vol='Garch', p=1, q=1)
res_garch = garch.fit(disp='off')
print(res_garch.summary())

## 6 Diagnose GARCH residuals

We check standardized residuals and their autocorrelations to assess model fit.

In [ ]:
std_resid = res_garch.std_resid

plt.figure(figsize=(12,5))
plt.subplot(121)
plt.plot(std_resid)
plt.title('Standardized Residuals')
plt.subplot(222)
plot_acf(std_resid, lags=40, ax=plt.gca())
plt.title('ACF of Standardized Residuals')
plt.subplot(224)
plot_acf(std_resid**2, lags=40, ax=plt.gca())
plt.title('ACF of Squared Standardized Residuals')
plt.tight_layout()
plt.show()

## 7 Plot estimated volatility

We plot the estimated conditional volatility from the GARCH model.

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(res_garch.conditional_volatility)
plt.title('Estimated Conditional Volatility')
plt.show()